In [ ]:
try:
    ! pip install gmaps
    ! jupyter nbextension enable --py gmaps
    ! pip install pgeocode
    ! pip install geopy
    ! pip install haversine
    ! pip install folium
    ! pip install pyproj
    ! pip install certifi
    ! pip install googlemaps
    ! pip install polyline
except:
    print("Something not installed")

In [1]:
import numpy as np
import pandas as pd
import pprint
import gmaps
import pgeocode
import requests # HTTP requests (GET / POST)
import datetime
import folium
import json
from pyproj import Transformer
from geopy.geocoders import Nominatim
from haversine import haversine, Unit
from IPython.display import display, Image
import googlemaps
import polyline
from datadotgov_apis import *

pd.set_option("display.max_colwidth", None)

In [ ]:
ssl._create_default_https_context = ssl._create_unverified_context

## Helper Functions

In [2]:
### Test code ###

address = "Bedok South Avenue 1" #user input value - destination location should be HDB area
postal_code = "469334" #user input value - destination location must be HDB block (example is Bedok South Avenue 1)

print(address_to_coord(address))
print(postalcode_to_coord(postal_code))

(1.3220073, 103.932284)
(1.3168, 103.9324)


## Set Your API Key to use Google Directions API

In [3]:
fileObject = open("apikey.txt", "r")
api_key = fileObject.read()

In [6]:
#api_key = '########################'

In [4]:
directions = googlemaps.Client(key=api_key)

## Get Directions

In [5]:
def get_direction(start, end, mode = 'driving'):
    departure_time = datetime.datetime.now()
    direction  = directions.directions(start, end, mode=mode)
    result = {}
    result.update({'duration': direction[0]['legs'][0]['duration']})
    result.update({'distance': direction[0]['legs'][0]['distance']})
    result.update({'directions': direction[0]['legs'][0]['steps']})
    return result

## Display on Map

In [46]:
def display_map(directions, taxi = [], carpark = pd.DataFrame({'carpark_list': [], 'carpark_latlog': [], 'availability': []})):
    colours = {'WALKING': 'blue', 'DRIVING': 'green', 'TRANSIT': 'green', 'BICYCLING': 'purple'}
    display = folium.Map()
    lower = []
    upper = []
    
    for i in taxi:
        folium.Marker([i[1], i[0]], icon=folium.Icon(color="orange", prefix='fa', icon="taxi")).add_to(display)
        lower.append([i[1], i[0]])
        upper.append([i[1], i[0]])
        
    for i in range(len(carpark)):
        folium.Marker(carpark['carpark_latlog'][i], icon=folium.Icon(color="green", prefix='fa', icon="car"), tooltip = 'Available Lots: ' + str(carpark['availability'][i])).add_to(display)
        lower.append(list(carpark['carpark_latlog'][i]))
        upper.append(list(carpark['carpark_latlog'][i]))
        
    routes = []
    travel_mode = []
    for i in directions:
        travel_mode.append(i['travel_mode'])
        subroute = []
        points = i["polyline"]["points"]
        decoded_points = polyline.decode(points)
        for point in decoded_points:
            subroute.append(list(point))
        routes.append(subroute)
    
    for i in range(len(routes)):
        if i == 0:
            folium.Marker(routes[i][0]).add_to(display)
        elif i == len(routes)-1:
            folium.Marker(routes[i][-1], icon=folium.Icon(color="red", prefix='fa', icon="flag")).add_to(display)
        
        folium.PolyLine(routes[i], color = colours[travel_mode[i]], weight=5).add_to(display)
        
        lower.append(pd.DataFrame(routes[i]).min().values.tolist())
        upper.append(pd.DataFrame(routes[i]).max().values.tolist())
        
    sw = (pd.DataFrame(lower).min()).values.tolist()
    ne = (pd.DataFrame(upper).max()).values.tolist()
    display.fit_bounds([sw, ne])
    return display

## Test

In [16]:
address = "Bedok South Avenue 1"
start = address_to_coord(address)

address = 'Ang Mo Kio Central'
end = address_to_coord(address)

In [11]:
walking = get_direction(start, end, mode = 'walking')
driving = get_direction(start, end, mode = 'driving')
transit = get_direction(start, end, mode = 'transit')
bicycling = get_direction(start, end, mode = 'bicycling')

In [40]:
df_hdb_cp = carpark_init('hdb-carpark-information.csv')

In [41]:
carpark_latlog, carpark_list = nearest_carpark(end, 500, df_hdb_cp)
carpark = pd.DataFrame({'carpark_list': carpark_list, 'carpark_latlog': carpark_latlog})

availability = hdb_carpark_availability(carpark_list)
availability_df = pd.DataFrame({'carpark_list': availability.keys(), 'availability': availability.values()})
carpark = carpark.set_index('carpark_list').join(availability_df.set_index('carpark_list'), how = 'left')

In [47]:
display_map(walking['directions'], taxi = taxi[2], carpark = carpark)

## Save output as html

In [221]:
show.save('./route_map.html')